#  Language modeling with simple RNNs

In this example, we're going to write a simple RNN in MXNet using nothing by ndarray and ``gluon``. In practice, unless you're trying to develop fundamentlally new recurrent layers, you'll want to use the prebuilt layers that call down to extremely optimized primitives. But we think in general, if you're going to work with this stuff, and have a modicum of self respect, you'll want to implement from scratch and understand how it works at a reasonably low level.

In [31]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np

ctx = mx.cpu()

Ok, now let's role up our sleeves a little bit and mess with some data. I grabbed a copy of the ``Time Machine``, mostly because it's available freely thanks to the good people at [Project Gutenberg](http://www.gutenberg.org) and a lot of people are tired of seeing RNNs generate Shakespeare. In case you prefer to read Bill's poetry, I've also included Andrej Karpathy's tinyshakespeare.txt in the data folder.

To get started, let's read in the data.

In [32]:
with open("data/timemachine.txt") as f:
    time_machine = f.read()

And we probably want to get a taste for what the text looks like.

In [33]:
print(time_machine[0:500])

Project Gutenberg's The Time Machine, by H. G. (Herbert George) Wells

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Time Machine

Author: H. G. (Herbert George) Wells

Release Date: October 2, 2004 [EBook #35]
[Last updated: October 3, 2014]

Language: English


*** START OF THIS PR


I went through and discovered that the last 38083 characters consist entirely of legalese from the Gutenberg gang. So let's chop that off lest our language model learn to generate such horrid screed.

In [34]:
time_machine = time_machine[:-38083]
print(time_machine[-500:])

eaping that must
inevitably fall back upon and destroy its makers in the end. If that
is so, it remains for us to live as though it were not so. But to me
the future is still black and blank--is a vast ignorance, lit at a
few casual places by the memory of his story. And I have by me, for
my comfort, two strange white flowers--shrivelled now, and brown and
flat and brittle--to witness that even when mind and strength had
gone, gratitude and a mutual tenderness still lived on in the heart
of man.


## Numerical representations of characters

When we create numerical representations of characters, we'll use one-hot representations. A one-hot is a vector that taked value 1 in the index corresponding to a character, and 0 elsewhere. Because this vector is as long as the vocab, let's get a definitive list of characters in this dataset so that our representation is not longer than necessary.

In [35]:
character_list = list(set(time_machine))
vocab_size = len(character_list)
print(character_list)
print("Length of vocab: %s" % vocab_size)

['q', 'c', '_', 'f', 'v', 'B', 'y', '5', 'M', 'g', 'A', 'E', '#', '?', "'", 'k', 'L', '-', 'o', 'O', 'd', 'C', 'K', 'z', 'W', ' ', 'x', '"', 'r', '(', 'H', 'R', 'G', 'X', 'p', 'N', '[', ';', 't', ')', 'l', 'm', 'D', 'F', '1', ':', 'a', 's', '2', '9', '8', 'I', 'J', 'u', '*', '3', '!', 'b', 'n', ',', 'S', 'Y', 'V', 'P', 'T', '.', '0', 'w', 'j', '4', 'i', ']', 'Q', 'U', '\n', 'e', 'h']
Length of vocab: 77


We'll want to access the index corresponding to each character quickly so let's store this as a dictionary.

In [36]:
character_dict = {}
for e, char in enumerate(character_list):
    character_dict[char] = e
print(character_dict)

{'q': 0, 'c': 1, '_': 2, 'f': 3, 'v': 4, 'B': 5, 'y': 6, '5': 7, 'M': 8, 'g': 9, 'A': 10, 'E': 11, '#': 12, '?': 13, "'": 14, 'k': 15, 'L': 16, '-': 17, 'o': 18, 'O': 19, 'd': 20, 'C': 21, 'K': 22, 'z': 23, 'W': 24, ' ': 25, 'x': 26, '"': 27, 'r': 28, '(': 29, 'H': 30, 'R': 31, 'G': 32, 'X': 33, 'p': 34, 'N': 35, '[': 36, ';': 37, 't': 38, ')': 39, 'l': 40, 'm': 41, 'D': 42, 'F': 43, '1': 44, ':': 45, 'a': 46, 's': 47, '2': 48, '9': 49, '8': 50, 'I': 51, 'J': 52, 'u': 53, '*': 54, '3': 55, '!': 56, 'b': 57, 'n': 58, ',': 59, 'S': 60, 'Y': 61, 'V': 62, 'P': 63, 'T': 64, '.': 65, '0': 66, 'w': 67, 'j': 68, '4': 69, 'i': 70, ']': 71, 'Q': 72, 'U': 73, '\n': 74, 'e': 75, 'h': 76}


In [37]:
time_numerical = [character_dict[char] for char in time_machine]

In [38]:
#########################
#  Check that the length is right
#########################
print(len(time_numerical))

#########################
#  Check that the format looks right
#########################
print(time_numerical[:10])

#########################
#  Convert back to text
#########################
print("".join([character_list[idx] for idx in time_numerical[:39]]))

179533
[63, 28, 18, 68, 75, 1, 38, 25, 32, 53]
Project Gutenberg's The Time Machine, b


## One-hot representations

We can use NDArray's one_hot() render a one-hot representation of each character. But frack it, since this is the from scratch tutorial, let's right this ourselves.

In [39]:
def one_hots(numerical_list):
    result = []
    for idx in numerical_list:
        array = nd.zeros(shape=(vocab_size))
        array[idx] = 1.0
        result.append(array)
    return result

In [40]:
print(one_hots(time_numerical[:2]))

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
<NDArray 77 @cpu(0)>, [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
<NDArray 77 @cpu(0)>]


That looks about right. Now let's write a function to convert our one-hots back to readable text.

In [41]:
def textify(vector_list):
    result = ""
    for vector in vector_list:
        result += character_list[int(nd.argmax(vector, axis=0).asscalar())]
    return result

In [42]:
textify(one_hots(time_numerical[:40]))

"Project Gutenberg's The Time Machine, by"

Great, it's not efficient, but we know how it works. So we're already doing better than the majority of people with job titles in machine learning.

Now, let's chop up our dataset into batches of 128 characters.

In [43]:
seq_length = 32
dataset = [time_numerical[i*seq_length:(i+1)*seq_length] for i in range(int(np.floor((len(time_numerical)-1)/128)))]

In [44]:
len(dataset)

1402

In [45]:
labels = [time_numerical[i*seq_length+1:(i+1)*seq_length+1] for i in range(int(np.floor((len(time_numerical)-1)/128)))]

In [46]:
dataset[0][1:10] == labels[0][0:9]

True

## Recurrent neural networks

[Explain RNNs]

![](img/simple-rnn.png)

## Allocate parameters

In [47]:
num_inputs = 77
num_hidden = 512
num_outputs = 77

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxh = nd.random_normal(shape=(num_inputs,num_hidden)) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whh = nd.random_normal(shape=(num_hidden,num_hidden))* .01

########################
#  Bias vector for hidden layer
########################
bh = nd.random_normal(shape=num_hidden) * .01


########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_inputs)) * .01
by = nd.random_normal(shape=num_inputs) * .01

In [48]:
params = [Wxh, Whh, bh, Why, by]

for param in params:
    param.attach_grad()

## Softmax Activation

In [49]:
def softmax(y_linear):
    exp = nd.exp(y_linear-nd.max(y_linear).asscalar())
#     print(exp.shape)
    partition = nd.sum(exp).reshape((-1,1))[0][0]
#     print(partition.shape)
    return exp / partition

In [50]:
softmax(nd.array([10,-10]))

[  1.00000000e+00   2.06115369e-09]
<NDArray 2 @cpu(0)>

## Define the model

In [51]:
def simple_rnn(inputs, state):
    outputs = []
    h = state
    for X in inputs:
        h_linear = nd.dot(X, Wxh) + nd.dot(h, Whh) + bh
        h = nd.tanh(h_linear)
#         print("shape of h: %s" % h.shape)
        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear) 
        outputs.append(yhat)
    return (outputs, h)

In [52]:
state = nd.zeros(num_hidden)
print(state)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [53]:
(o, s) = simple_rnn(one_hots(dataset[0]), state)
print(o)
print(s)


[[ 0.01300892  0.01294263  0.01282525  0.01310373  0.01312327  0.01299322
  0.01327234  0.01308019  0.01301843  0.01283188  0.01279322  0.01302909
  0.01279998  0.01301603  0.01307486  0.01324012  0.01310832  0.01296501
  0.01281886  0.01304485  0.01314966  0.01318836  0.01282357  0.0130063
  0.01319014  0.01289848  0.01297821  0.01306884  0.01284166  0.01300059
  0.01293945  0.01292841  0.01306092  0.01286195  0.01318577  0.01293136
  0.01318113  0.01286769  0.01284658  0.01297056  0.01289557  0.01312511
  0.01312432  0.01301217  0.0128699   0.0130516   0.0130927   0.01305016
  0.01295495  0.01289698  0.01296742  0.0131414   0.01281217  0.01297734
  0.01316937  0.01305398  0.01306568  0.01305074  0.01288696  0.01282803
  0.01308305  0.01302677  0.01289196  0.01292999  0.01280847  0.01300131
  0.01304248  0.01271127  0.01286089  0.01295225  0.0130857   0.01280189
  0.0127888   0.0131125   0.01290075  0.01315066  0.01281501]
<NDArray 77 @cpu(0)>, [ 0.01305367  0.01297818  0.01293298  0.

## Cross-entropy loss function

In [54]:
def cross_entropy(yhat, y):
    return - nd.sum(y * nd.log(yhat))

In [55]:
cross_entropy(nd.array([.2,.5,.3]), nd.array([1.,0,0]))

[ 1.60943794]
<NDArray 1 @cpu(0)>

In [56]:
def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = nd.array([0.])
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
#         print(total_loss.shape)
#     loss_list = [cross_entropy(outputs[i], labels[i]) for (i, _) in enumerate(outputs)]
    return total_loss / len(outputs)

## Optimizer

In [57]:
def SGD(params, lr):    
    for param in params:
        param[:] = param - lr * param.grad

## Sampler

In [58]:
def sample(prefix, num_chars):
    string = prefix
    prefix_numerical = character_dict[prefix]
#     print(prefix_numerical)
    input = one_hots([prefix_numerical])
#     print(input)
    state = nd.zeros(num_hidden)    
    for i in range(num_chars):
        outputs, state = simple_rnn(input, state)
#         print(outputs[0])
        choice = np.random.choice(77, p=outputs[0].asnumpy())
        string += character_list[choice]
        input = one_hots([choice])
    return string

In [59]:
epochs = 300
moving_loss = 0.

state = nd.zeros(num_hidden)

for e in range(epochs):
    state = nd.zeros(num_hidden)
    for i, (data, label) in enumerate(zip(dataset, labels)):
        data_one_hot = one_hots(data)
        label_one_hot = one_hots(label)
        with autograd.record():
#             state = nd.zeros(num_hidden)
            outputs, state = simple_rnn(data_one_hot, state)
#             if np.random.rand() < .2:
#                 state = nd.zeros(num_hidden)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, .01)
#         print("Epoch %s, iteration %s. Loss: %s" % (e, i, loss.asscalar())) 

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])
      
#         if i%100==0:
    print("Epoch %s. Loss: %s" % (e, moving_loss)) 
    print(sample("A", 256))

            

Epoch 0. Loss: 3.05328143644
AA.Bueamllieet ogth dnal
 hehh hn walheabptd irli.iuha ,iauwl
aeshtptnmhhr fudedptoon  ctsinwot anpd-bmTiet
eehegsgritpistea
a nsng uue 
w   ih  eerhasd lt  ibayeoy scdmaidhew .shuul. i,polepuemIh evmshnnmotiotho ihtoi uaw e ug wtoi Oetnii h iointu egkea pt
Epoch 1. Loss: 3.03976502278
AHwgimeeelt amesihhfol  dt nhiouda
alggpudhQgiagidf liht hvsdh
  oahlig hneeg.slsi  ce
e h tiot ehhaut ps,ogfiaer f8ehee sieaaeteiedr h dmnmdmseonwc  o d fouhhahn ytnit  ars mrlwnet dnunn  hti gait nbbeltt o i aeyt , hlt hefft aE 
cah,  ei tmp eetliolrecsf 
Epoch 2. Loss: 3.01964210843
AwYtdcran  ot eireytmand aalentieiw ownlpiisctgoi hwtu.iimiemn
ecmtsmer at dds plaann cd imkwdoaemlmfuet t  tnmsy nkns eoteutt so aesnoeIesihd smnhtmy o wl   sin bkvhi oepm a tftuewoei ihheu etek5nri
rAhahunccrdboiT a  umtao1Iht  wapurrhn prsooa asalga aote
Epoch 3. Loss: 2.97754519845
ADuc1mo  iastt ebd,nafmg mdwhe YodI i u atlnned me nerleiayint  oiuh  hhloafi talnm  oher nortoAl ase  aTfehuc

Epoch 29. Loss: 2.26114134365
Ann;igkdinny ald at mthon thiga tupnkellyit soted chokingin en.
gary _Hithel. hae .acFlongopegetle wate audwlickt worginelyited tuy towe cares thatured and thetuen, mhenedd,red tubel theracparg. The and mace stewerasrrofidls. Igelkitg. I' fiaglaly whe pangi
Epoch 30. Loss: 2.25227259644
Alle l ionionift and trous
dam. grcakaud ine tiouto melist
. This ofatlived iofwat onrciw tont mbyind, wroat meaniculastoning he
thale n chet tavold fke sowathen weranguthen

uftomh, f ther of thit hom mates kerandiand gpeimenonoungand aspwor, ast wesaigir 
Epoch 31. Loss: 2.24372101257
A Thet to whe hpaverle-int
persy poo fonttisited the pamer. Iuwt wimat dame sseame towowledptreds hee nwpdiles an ma faintesH fweptind be meme
wche weathro pasuistinturey ip atiming awe, alplulw
thec ofitstou ilpt osde
thruswithred semedind seouedcanding ot
Epoch 32. Loss: 2.23544597981
A0tius mzcand w totesutthat sy wiwthy ditly pmitha Turtonastobtterd floun. The diwtrepsellyyuticllot wead,

Epoch 58. Loss: 2.04342846292
Aving the intist. Thee?d th ardidentoon fopthand of hoved room dofked, on a wothint of and pleaing sotedioge fom boungen of bhitht dinbluegs. And wark foutuig pupeat. I way thiskibitay capedote plomention mitho dreat of fuss of whas ins anw'redod-ly int dit
Epoch 59. Loss: 2.03626674676
At. As anpint a unothintholo,' stowery moremalllyob thered thowst, qpocent
of
lycicunoo thiouss fad yand dlontincless wiraf Eired'uodato f med my
mank.

I
withre povares iy to the thawise fur moupist. Ar metien ly
theree, bly. Ih was il fare and so furs.'
t
Epoch 60. Loss: 2.02912403224
An oun moment ij
Tume ttaveryy is frofirg. in wheb, a aypathed, shhinteclly neaingof meramd anditacingenwy cutome to mene pverevela if il fthintting fade cfomeds orzancispep.
'Serdinted thichenngo cuppetidisirswanded bound thee mofenparssing-uplasury, indab
Epoch 61. Loss: 2.02199637708
AQr bares nenetade pollyniste thja:we tapares on outherofeling. Then the eadring eruut mowhisted foots anv

Epoch 87. Loss: 1.82629937675
A tiwt ingupperinatuly feltins of is treme weading ow a sbout dives futmys bearn sswinge tu
ceered a calminment pithest yow or werariaging manghe me
teen gawh ishodimal? Bht ouscese and mas into mpout fithed that thablle thoured foon of ilmur
madlde!t I wey
Epoch 88. Loss: 1.81678582181
At mo froulds wooted in kbom throled tithing foomone foom to There arisstooneder, beed for was ffilareven bycanlest: and
taly
theal tergme, to fur, very wistudenmys tcing in one muchune;s ouc to malinitary ay wegestred; the treal in thouck cammy men with a 
Epoch 89. Loss: 1.80687617307
Ant stain fit to mechided.
Huraysiousalling
eremy the and I enothorghine.
A gad me, wht pthad meat effly dinuch of hou se
-inelly moub. And sowhedse
ke han me groun pled-hed nopars in
whath othiog the
expan thow sheed, and mawes thit thrast fowand petarinas
Epoch 90. Loss: 1.79648299931
A dalicep if moin ersstised the heathing at ant crealimessmonc. Then thew sictrens thimpecuf aid tham che 

Epoch 116. Loss: 1.51601775515
As greyted the lecthrever not tf intoraicent bailed sare pleapeares world no dimon,
inta same of at and mo geraplestoun out clioklly on th had suning in my.
Ind inmo thet mich the withone what agd chen gotertime shaw thearshen of with no sigh war s apof ue 
Epoch 117. Loss: 1.49777332584
Am neg
to l theearn wesile at wh the and ow th was to
me se.
W'w
rvirutred care the
mate and darcing mit ofe my
and lid. The

upsout. Theme fres thing and dewing with them to mocainl and froundssion.

'I nay
humperf indons?ory it to me or a our
kr wate a fa
Epoch 118. Loss: 1.47247603421
A doons would neq it on were than showers myappped lyqout ce. his interputime whon thevel the sade!s of eroutnmes fe the starked en tole monerrep me plould leighed foo jert a at
intenttor. Ihe

thabr alm.
It'd of intu theillerting.,

'Bou bleachine they uro
Epoch 119. Loss: 1.46024644194
A wrile whit for the
hearly uldaps, and simans in
ous like the ghacking- fald itting theve cheared amy

...

Epoch 174. Loss: 0.865691795267
As to
mely tawards me, ink
abd and dod watf roster on y
yoince fad chamilutive yhat growher four, we cunill of the inter:ainedisabler. A fid mes intly wert craid brantll win wait. Onot wow
shinning o
 apl and the whole my my mystlf grounndlissoviny wnother,
Epoch 175. Loss: 0.918154739322
A wasirgain frackerecm It pact a tire leagl at in cenile
fibly of mevenlitife who gas thair fol it thome lesp shealled lithing theil fresifations. But blooress sme llooghourhold. Then the faum. he was that
lughat with velursee
fou sail the Thaim grest; itte
Epoch 176. Loss: 0.87080653491
A sumpered
unseline
blingily apor. It my listertat me.

ant rook bete. Thirk wo linge, a wittre w sthen poow rist luy co the mats ow trear sofidite,

ruslt,'t meng gaish-ablerd wis leadly spergal ffow in the
welb-ofw tustuld follithentile

Waveres a mast th
Epoch 177. Loss: 0.914571394132
As tliggresh theild be
thend ene sthond with bever a waster. I wowg?ld the edouples. The seming to


Epoch 203. Loss: 1.39707055322
A tho Eadest of thism chishentlssoum clint on themy in thered stoodels pechence was
not coondintowen
whok twey. Ime wime that sheirs. Hhop wist choored, of pustony clotwerldy
waile an a henthod. montw accessiladitper-stook now thements wet of loose.
Aponcth
Epoch 204. Loss: 1.3840786023
As
bThen the ldomentlestentl faittent
tinnthor. A fountlesp there dlmit all what im the same to youron whouenty, whroutd mas this got tort, cblokint ave bren floch. Adtibilanishentcint andscconstt blint craithrlbtinn theme;plest can slant rettinnstorabs bet
Epoch 205. Loss: 1.36474469383
A pur time travelly
lantit we md, oat ofirclinas
shinile filliltton, sur ourored me it
intichrewnnt wor come thoowerlly unithingethevinitily the thingingat haid. mo towe mowatterinaty itr epresite be cvet ut. chey wete tarm to
autu ot ot attens tobe. Ind ow
Epoch 206. Loss: 1.34372531505
A sar our me is a looder fade. They melecthonest mastor a sig
thomant anticimation, creas macter. Then 

Epoch 232. Loss: 1.08932063286
A coulunit jack explakest a fromon be loored atble
with tr a gutions that
lyometeseble smoge thited-nith--Bus
fuce the
liks wo
my, and came wingly were the fure to bountivel that thes geen at sowe fiscout lo-st, and the wew of anousent clutels the peoul sot
Epoch 233. Loss: 1.05804822729
A doO of a
toue foos
peaut
of a mappine baws
and co a wind of cuture th ig hoss of a mich to the peto theind, and sarail to lecolf fromod lo bemundent brosmonhetherd all the She tiness. The gees wer tows altaring them wowawatiots one sawadd
backeds wan smee
Epoch 234. Loss: 1.06136139175
A doin thein it. cados winbuld store thith the
wad of a amounpth,
and cresently sig to we thimg with a wastening the
chomovilb--theingling Rut

a
thanding of them, acl time to me mas bainte mughine furt that wauted to me prain rloning at each theind wan tho
Epoch 235. Loss: 1.06424882458
Ablu. Wlither. Bhisgem thoubd thaigh the helorito at a pith qhion.

'Ascemilind

to saining mid bo at 

Epoch 261. Loss: 0.881630192347
As oxI corly ofenticclfaim, aponcto lave all willd beonith of whice curare of there warngat, foighthor to a sight of wluch hiven were varerl feilitt
of
porer of to the shoulds fillofiond the
wers dreweectoom the punker for ackear of ould
seocuntyeot of it f
Epoch 262. Loss: 0.918450808705
A doink'
but treether mekented that dreyte at appeaw a dryenalizalas megrise layk mo the Time Machine
any coritide, and man a with trist ond these chem ous of futere furtarily in myssDuend inpoctine thir sough dais. The seem a witn that me. Then, which was 
Epoch 263. Loss: 0.878811406332
Andy-wow y.'
FBurtide twey mas loome thet moourning soue like the thaugh stintiene weec aw on mo eas of bucure itheryw were laverw there was not gook who camcous fite the toomity gat hearginas doowing upsimessone
way. I wean a siok bee
theningwazt wot hiss 
Epoch 264. Loss: 0.905447286569
A doinug
benprowe take exLeysene of them above thit to sectey ley camation it
refallict in my lwis

Epoch 290. Loss: 0.747704500188
A doinsclossuckee fore sheir flob un mistine mastered of meadding of a cead frow thee
wigh of mishove

A
ke
thing to
mak trew
what whe
pred one of
audomentirused to sad bur
menticipt in face sargle and will sheaushed my hat stommered,
who was remotive, of
i
Epoch 291. Loss: 0.804586278457
As ouruding Miclly
hok-.1AEthus. Tr
carky hore, (f-dilu!nYly sorexn.

*xarlyy fiablyV--D'Mmy..
HeBrit,'"A*2Mgrzc,'YAghald.
HWr-Drrend;MbHOhel),'
s,orexy.'AVMonMlqMe.'Thrviry.
P'My hixghedr-C
tooxknYMr, IxcorW. Mrrell,.,
Hugry--DxMedMl.
TWmblykd hovesD-'Y
'Y
Epoch 292. Loss: 0.808219123511
At mottle wooted to fertrusers.-I sappereld utt spumense himed to me, beme
them machine in bossowing in sowh cume, and prosens. The Theee
there wete smeative the Time Time new of a shined trat goticist thich one me, chenofe!
were derow, mut moven, them. for
Epoch 293. Loss: 0.796681804454
At mattle swarss, and latgestomblersmodives. The while wapl of owht
munss off white sumentenist ma

In [ ]:
for i in range(len(dataset)):
    state = nd.zeros(num_hidden)
    data = dataset[i]
    label = labels[i]
    data_one_hot = one_hots(data)
    label_one_hot = one_hots(label)
    outputs, state = simple_rnn(data_one_hot, state)
    loss = average_ce_loss(outputs, label_one_hot)
    print("i: %s" % loss.asscalar())

In [ ]:
# def sample(prefix, num_chars):
#     string = prefix
#     prefix_numerical = character_dict[prefix]
#     print(prefix_numerical)
#     input = one_hots([prefix_numerical])
#     print(input)
#     state = nd.zeros(num_hidden)    
#     for i in range(num_chars):
#         outputs, state = simple_rnn(input, state)
# #         print(outputs[0][0])
#         choice = np.random.choice(77, p=outputs[0][0].asnumpy())
#         string += character_list[choice]
#         input = one_hots([choice])
#     return string

In [ ]:
sample("z", 128)

In [60]:
####
##   Copied the train block so I don't lose output from above
#######

epochs = 100
moving_loss = 0.

state = nd.zeros(num_hidden)

for e in range(epochs):
    state = nd.zeros(num_hidden)
    for i, (data, label) in enumerate(zip(dataset, labels)):
        data_one_hot = one_hots(data)
        label_one_hot = one_hots(label)
        with autograd.record():
#             state = nd.zeros(num_hidden)
            outputs, state = simple_rnn(data_one_hot, state)
#             if np.random.rand() < .2:
#                 state = nd.zeros(num_hidden)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, .005)
#         print("Epoch %s, iteration %s. Loss: %s" % (e, i, loss.asscalar())) 

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])
      
#         if i%100==0:
    print("Epoch %s. Loss: %s" % (e, moving_loss)) 
    print(sample("A", 512))


Epoch 0. Loss: 0.506973500355
A doo fot seesy a dint me.
vound fike the were more when they aprest andire the some
surabe warl hode more deaclo to the madger towo spyond moted. Thought of as on
mate pur
heve be
lave and their lear; explain in
a the that before wert 'voouph toundiding the poughar; at lass cheir feriddialy-mater-z
ffally into their leatere drmels a ang lifice thing on imoudders was not I had begenterean be oploss in of a gattered hil co fule the lough daways gereaverest a
moring and with a wasted that theinged the atce can
Epoch 1. Loss: 0.444907182016
And3 begiry.)'Hurys worer!d,'s
rxss.'TLFry ixcroutloughel mxredrd,'I
grer-wh.
Wxmory. MFmy.'Th
NAglok,
Hed by bor,'s
mor?les;.'
Hor;SYMy fopt.'Th,'VrullW, tormanc4Thed sich.
LMMask.
MWonexc'ThXor's
DBrork;,'8rraulhow,
TAUSY.'
'Wiglork!'WsTyI 'xMer..
Hery
HxceDux.'Th'N_ by Mot,'C
Top rxw, Iur]. Sxy or. YxS4y hover wnjSswine
ry.'Y'Pgres. PngrX-,'A'
Saris Ixu2_Yors.
Tind,'B(g, Tryo!l. H*y ppo?'xhMars.
Lond:reDbllyM!. Wxy utar

Andy way Fxrelvottor
ver YouckAurYmySchorngho'Wh'shor shyX'M
loMs hor,.'TrFs Ixs,'B Mor bext'Yfurex!'Mspor.-Y5Thest.
AxL'ly tor
hoplly Figerswinh
ofy ro#l.'Annofrous
YTinv!y
shadn?'Medryor torerd;e'Pex'2.
PWronk.
HQCrest'CAQTreso..!
Mo_j.
Thurfurhudr;y
hedd w.
IThany.-
IXrert LxP5_B, spyovexgr8or,'
Limy,'
rAchrred--'S
Soxmed!'Yorysivh.
Whal wxy-d,'M.'Medr fxy?'O Mor Txrinver-shorexplong furmalex'Y
trory,' oYThMs har9inx!'DrYorex. Yorhesprent,
byMe..
M'Mllight
WH#S_xt'
xMmore.

TDmy?'An'. SorexBs,
fromy.
MxMe
Epoch 16. Loss: 0.269573934766
A comericull of into suge at il
hosp aith buld in
mysented tr esher me, intustly dimention look. I fooke lyomeshup nothor, and a sare lay bee maushod bo ard ext gutorvarable become hissed me, Net seen uas the in
and lofeest ripeney cotose with a scapp and sway suglost a grack on his car]ued
to fade if read; on eaps
condised briad hasten the lecouct not come what we see) ut a wentiond. And woure bumenttC-make seeges, and with
bevele thilk out of seem T

Epoch 31. Loss: 0.228387210939
Atr,' coring of anyes on milude of a thaif genpleerd a coudd sone to iscoressed firime tanis me
litelb unhation towards the mookm hid byindeaugatdea neto epof his becement badede the pees at Slace, and thene in. At for a moment hefelts, fulloss of foik luggrituse that woukd folleds atare werime foould began hime we hil gimp, and the sofems anding a sard uillingly freely the face was forg.'
I'p and, and trow to fiening behinglat in
them like the odes wirh lisht ond co exteksucif reoled woild be ear copaid, an
Epoch 32. Loss: 0.219412017772
A doo gote
frangsmend
that, Dliot: ngation of porom ther undomerain?'

'Eher indostigu? WHateed and prefly for weve fartice frum feefivaly it furnds we let with me mosild, and the ressur donseale them the peenot?" saiftentigy, the Mevoly untel the rounted lade sooulared them. I wooked here
putiling
intiretable upsain Ravere then the Eemore it ag
ond, fould de ppoind through ther, when the Tire Madhine hearts ppoonch the 

A doo hotse
fortary mitule foot soiftle; fur
thes. They arofe one
wonged a roure, a tlushing
with this reoor have ex
onting to
istloss. Oave sevel epee
the sud hain, (row bepureac was growite
ulankesseves antive of lime, a pright le telbriteen a clamp al
thine mantecreation rererle Fonding be anling, nou aloot heo Neat turickede to the sur. His gecked actitiness of thery wall
igrulare. We hals were foretigh, I bevel trailled rasker syhare in prounded and cuerid hadlly at them sodion to streathe me savees, th
Epoch 47. Loss: 0.195580819575
At oU chimughe him I rame sunding bef
corfock ow punding surgard, with a wmsthoud fit ward I lege mu
minuse bay comf accoust. AAd cheinginger corta serong ittont of he har meang the Henste in a qurtly the od the recowe for a simple sthough ham

Now a had agine the trust was ol a shoke as an atcoug gut us me his war
urd in my hand dree od the fromented that
to gress deaver launs betenved that the Lem in Gyoune more them. So I shook le, broshed that?' p

Epoch 62. Loss: 0.166666686421
A doo sof wo dratee uphtod secone hime. There eas ackess--I relo the full of lite fall longr, withoutly, and incinting the more
seem sevelfle bligala a fis dees for shend reanting. They unemmer atm nevogutey. I suppose whiles-oved at each dawnlogg. I mone, I fain lyselity in a thustho my hend, andeding in ughing sollorg the morem. They hear as it
mbaning filly, and so genthem wound to oress, the at ears coustle. You paly, and their feats
of our pracesuragehys whunsedsup, into mimentionelled and I suldest one
Epoch 63. Loss: 0.174086650472
At miture spbeited and freg
ingevarateenzat and him inting. Et
enfo way hare abrectly strenfired be able trave dosen, at
one of theme bealtie. I saw of fired
mard; and with, was excesp itt "ound for a moment I fag seamed into
their fuated od a teroc me, the remo rim to see
was leadilation him coustenty, a thud frenz lith daccloald. Bhe
gens
werc I men shows of madees ffow by cirtoon?' said the Thies nop tree with baund w

A doo hovjus. I dow's grew vet the sud,ing of poicl we
thom stonevery it to mede fasmyrist of Fent. I dow's farg. I was se of the firece a sigole a
man!ed sigh pote consufter-ugain they med that do were
some of ond his. 'A took wazched by hild-ib befone he was meven of an there is no ght rough, and paccing aboup
the Time Machine, but sifes of corrawite in grested that the pactian the withe:t one has woted the ker slowe? anticinat dreeness them way agreall on the in all tentlese our cumsly jetond menall with 
Epoch 78. Loss: 0.146025966937
A domerd.'

Then ure, crutter, I had some slosping to my ears whimg.ed a wny mpans grey fuinting
the ofos anong on with mithing my ablo ge that rind to dinger it a plain int, intorther wond's onted the Pderoct on theme exalting the bougt mend painds on my was ganges of me arqols hus werealedens flation lf the mighading and courder. It
lass chit clain meneat of with a lear on mysting of the moce, and the sapth ur Det courthing stolend on the
Dad mone o

Epoch 93. Loss: 0.130610873228
At ourthinzs, and for the

srear that
wand the bell thang. The tiressaven some
puixt grawe Etine aghis ngating in a qubule there was a certablece his hopse
. ven a surfer to asmone


Yend his been and whe hald zarnge betatious great; of them
about
little vesued on moge at alf and came his wtyes toword An whim ladge.

TEncenens, hind of the ipprated on the fire, then cluthis flow the twinger.

'That in me
toward, and gangul I burhed the werm nith of his bechinilaring at ghe
eees wf cructingss.
Teniarning whit
Epoch 94. Loss: 0.12126914658
A doo rot me till and piming skupted it of the incenternlys my mand. Then weucaullokiver abd
rushing
enol sot,
come that upen it urowing tham the sed was in
withoutly at ey mactine had tree
lilt sufe bazg in a curfuler
waid the with send thee? I ampare sack and dil.
Be was liget on a crabof his flessed up a thaght how the hearee woting to mannew and, the meround stant. I das and sheak. He wall
shenting a tibe tenniglf
upo

In [61]:
####
##   Copied the train block so I don't lose output from above
#######

epochs = 100
moving_loss = 0.

state = nd.zeros(num_hidden)

for e in range(epochs):
    state = nd.zeros(num_hidden)
    for i, (data, label) in enumerate(zip(dataset, labels)):
        data_one_hot = one_hots(data)
        label_one_hot = one_hots(label)
        with autograd.record():
#             state = nd.zeros(num_hidden)
            outputs, state = simple_rnn(data_one_hot, state)
#             if np.random.rand() < .2:
#                 state = nd.zeros(num_hidden)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, .001)
#         print("Epoch %s, iteration %s. Loss: %s" % (e, i, loss.asscalar())) 

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])
      
#         if i%100==0:
    print("Epoch %s. Loss: %s" % (e, moving_loss)) 
    print(sample("A", 512))


Epoch 0. Loss: 0.17398204026
A dxo Itrer to a groupered with the tolnicl jithonces evanct if a manucaire. Ind me. I couct frem the screw not,
mpont becumen ont crmathe sheaghed a dree fact rang the Tholow of actime, and the somethed but wast cuper and shallocaly now be any piatity pepomed:e, and thack mathinels appard the while figure or toctusion, that the simely an a gromot like tane conficeny then ackepted and four
bryugreed on the launen byon. The goom. I the trant nentid fook my
ceetree somether with the
brigine fulloked radfer and
Epoch 1. Loss: 0.164460683721
At me metisuse anp grey
vere ham. The
thentire wor
mole thing hos
nelanted to When wered he here all think in a ctuing and ce tach tee
think almeary unta istj and that? I remarked in them. So bust in a little prislllok has travelly the uprening, to me that the one
with tuth citaunzest. Time, shis bauy io flomens of the machine hour stoom a cramp in a mitule the white soughation have a rimulare his wett one of the othury, I 

Epoch 16. Loss: 0.147715801402
A doo hot
meaning
beluce, when whithess of whr
machine, his cound ftyued to wath a more gut
in which hys
mectudelt I fayt was
face. Sbout little wimurl at for a mitule on them
the cenalesu his gens
intervalsed inco the dourned rou wer
meauted mo exshoil..

'Mone bere clathiry itr staly Frel, sit. modevt ol a simple remiched mpon strock, and fourd the groscon its onl rent foElors, hapf at there was inkry larely otherstwhespess with thinge--he
tives acte the bullts of the waw a beail whe berove yoc I had come 
Epoch 17. Loss: 0.146681143711
A doo rot frown duaces efuiled foilts uh 2eet
ontigution-'I ore intorfils of burane so dontr, I caver the Uiref a prass when ipor the.



'I
sund nex I wancesune beew fiaccly the were lave ithaingathed no
 'Nuraul canch a
veather fouldsmed at whe or shee with a lmon Nos tacc for hism--tod, and sither fored my sexfif in of the Pit lopmon viret masd forgh my, thon by cain of itt ascead a doost one glaven it wht ntt the Uac

A coousclorts, asmore
beop think out of a brall dinicannond at the werecl and round the kerlurating--handering muterain with a drovondit. I sare forll to seem develiming of aKy? then the
sed, thing har seem vitt laye sire of a mooll han seeaif ald that it mecal ound lakhiry, to sook bevinitht prosed ringing the beel ching agr pacring of thind I loserf on the
fickest got sugid. Hill of the had bestertald the ather
ouluning whe haw bo greve wasing evely the yem ghe thoog des comprecry lake have dore then tome 
Epoch 32. Loss: 0.147224793779
A cholschatlently uptel wor? Bounderstorm that wazy a moment I was sthenf furmage a shope, yount mathere wail in
a sarffert was decon at the Dick redilled the weros be
pee the farmec uapures-rooned him ticiled at the night twingle. They a
grast I fert evesscrial to thes?weradou, and and the sholo. It white show a went no jeaching the willessoved the har

romethed and bossed haveres, but me, that it and laze and with
vemb thing his wained nover me. The

Epoch 47. Loss: 0.137106907145
A doo toingh
nivemy treversitt my, wowning to
eres gming, this. I mo
entencite it a qive wancu ce. Thail whine the three dinging the whale ryselferhording, haichey been
shaps, and who sent fravely uedownong, whil they mod was to ty this legpice was fect. Alling al we mome anether with deling through the little levels this inceives tow stare explais of cestaid tward hore dime, brfugting aponither bire sholl, mnderfene hare blung monithing said st eer on themispowner--ad whith verushtoniby. But reakent Gut the
Epoch 48. Loss: 0.137808322128
A doo hot mean to the und cimaling ene. I
do lone mend mo. I hat he sed me. I wave a shoded you then fourzlly wanis me. I fould raych the wer
sore of the reto exarting theme intr mevele on mantt of a mind, and shat furstin it for meled onerede--utting--Khannshed by in
thins ond herping one of fou he wes mores bringien ind st. Sterict ald, uttinidabiglertmong. I falloning re tall the modenta at the
gene,. Be
presend mimen

A doink Me, in foot. The groch,
and fack of which I wale a himVn on the hause une mourd the qrojof away fars, and everoct me
out of as up an these crearvily
in vaintenthy the same of the riglovigg of my more farliclly pace illa upaing his luge, so forr
with a smill of filly to the ppone of thingl--whiny. The Time Machine gownith, tracplas skintine swhimm, they coull aboving.

He sume of four
my ments beour fultentudr with the suthamital you the
Then, wetlored to maver. Ather wot mo the sundle regal of thun I
Epoch 63. Loss: 0.128311273528
A doo Iot
kst of the crolly to the sumuton. It the Tiden at exclounde was on with
velooded thaik on the baught or a cippred himalotires. Whather--our a pirdry lay in theme in this wooled, Our cimely fane. EAt beet. SA feat
inst and ledold and
grew you?d tay drowh it withover bution tane soon a wat mines bagg inta mitthion ught wirc me man in was secoent veit you tle ret? kifuring tot
oreat that courscly. Oot
a ceat the kovemed. Then vencately-underved

Epoch 78. Loss: 0.123276921039
At ourremit of the passw it gosk
a tigater slesaie twark and actirine of the intenset us in this, a do ed toimed! franile, the or
thing and brain. Ho das of choure bure craif ce yack a little wands
tooning
in whices we wan dend litt tht ere. I dow I af hy forcan to exen think ove conding his uptated that the payseng thenu,' the war way lossiniling ont in the ond with valbout? it I sughesp and I sookent -nirken custout yount flackeararatee; sconct the our. I smy of onether velu st perhat winct-.


The fursed 
Epoch 79. Loss: 0.127221525037
Andy bor, wischary pxcrrawlyMy orjulho
ghis axf,resh'EAflory.

Troor-a
knark,
woriSghthel'surBredrnex.'Yor hivallMr, Iver
urmacy,'Asaryy.
WT ais,
aublok
thtore.
buroon why hacthing.

Hbrupling, beroun Fobly,'Yous row, Ier?'Pnoul.
Herelly,'arkex rerm. Thurh,'Lhark, haromnLw,'(X
ofrest twr_gPren routharhd
'Y. Yorexle,
fTherrd,wwrysstor,
Yxthrrind

Twhilex-lith's wrter comexBut,'s arionght
Ary, worland;
HWrr_ilimenlars,'Br


A chourd
missed to id,t sempefte, and prage. I stermad in hyod whocinelureding al a sigd the verm of prospected in ore direction at the Uime tamind, and wast his, butt if woode fyo
wrach,
wall-zor why mosth theored the andorch paching
in agotion a firp. I wolld yourte at the way along yfuplerhat all of a sabie.


CMoment.'

'TI te wast so
h'al to the mychinith of him) of the
deer. Ho loug and wisk a minuteve woild gaily fonll on the is oretingly
wanged that the firsalare
the firghinge thing ou 'more befo, mi
Epoch 94. Loss: 0.116446787939
At ourremby anled the
call time, and thought or a care to
apcertad as I traveculationelf jadenting heid yough IHeed ant it and ditusmenting threec--atcemst. I was face har ceryop ons in one dinging one lould seilly aple, what it agpiestive of the shared more turns. But all the stomatod, to a dincilattipc of llokel un iett one wast dour a tird I met of the tars on the warl, Inde slowg
res, and epauther before the wetentive

the machire, but bistins and

In [ ]:
####
##   Copied the train block so I don't lose output from above
#######

epochs = 100
moving_loss = 0.

state = nd.zeros(num_hidden)

for e in range(epochs):
    state = nd.zeros(num_hidden)
    for i, (data, label) in enumerate(zip(dataset, labels)):
        data_one_hot = one_hots(data)
        label_one_hot = one_hots(label)
        with autograd.record():
#             state = nd.zeros(num_hidden)
            outputs, state = simple_rnn(data_one_hot, state)
#             if np.random.rand() < .2:
#                 state = nd.zeros(num_hidden)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, .001)
#         print("Epoch %s, iteration %s. Loss: %s" % (e, i, loss.asscalar())) 

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])
      
#         if i%100==0:
    print("Epoch %s. Loss: %s" % (e, moving_loss)) 
    print(sample("A", 512))


Epoch 0. Loss: 0.115808200544
A doo toin creadfry switlling
ed. Ihe mad deen. HOirkess ofal semind holding the refilers spamples got cime ngais. Drel the badgert is the refore
incere itralled spooch disterall fistained toucke spanz.
Hushed sove she thour presty ud. High me mas in ant incthat.. The gree on them aby I remad of his ond had scemed through him this inceined.'

'You mely care hir fors with delacked when the orer cumingen
lvoing have he real in med illuroul But ipst timentzy patsissercaly to apdest istoread, and thee ave perori
Epoch 1. Loss: 0.115544300865
A doo hot me.n
the should he eet aimensed I, wash that it
wend frested some plys of the Frinting them. Hourd the undely on of in come witl gene: woly sode
pees at douctly to the tibes, to lesplastentarne-sige spoound bale surios. That the face of the spouch on y.

ilEesperid with tyocint, somen melictading.

up comous of white skipling a rexticed to ee. 'Ou he sentard the tho kimetoid befoin newhalf
igrensed feem vauning a

A countif it stose
proon my centained treved ad
ne has gotorfid stood a knother that the keed to the upoomenta tire Uacking. Soly poourmer. The wetes beon thind with a candous of the thing wand the wowled byoushend
way ins, now, sights revew to me,. But the stars, and he you the ledmove be campre.

'There was doted in has nearing a rearlf to dowctomed travellikghele
w,y I calf
lugg my byous bit twants
ud.n lyos, or owh igiofulaty a stronghe or
velt store like drans. into fillysife way in that the funprentary
Epoch 16. Loss: 0.108917367098
A dxo rever thre gone of the intonatised
and cimiding of becouninnty to
see whe tall to the spose of a moment and ears warll on the swarting lever wing in thehivermovingiry, trean ipserf bike think his leash fured ur attery pearsions werts ceopitall to the troms. Ynowghed a to denclinitations to ur seefed the lich
pactlit to
avcrot meating it tounted hamparse and
jound you thew foucing it twent copsepred that it sciocieyt troeg or a large. Aanding in 

Epoch 31. Loss: 0.102276328759
A doonveres, anlith itou the see in the cantienty the over collushed that the Premood Me?his fean Yishivation, the prosent turick white mid burscem. I had of the camp curest in simenlion vage a minu, and shey
mevelfervar bure I Bad he heve beered this imopproading of theme ber
mpringraby a limple sphinx, nowing of fwomite. I reqtover rimes. Tane hime sove ally madding
should chough dolf premff the
anew who doskits of the creeting like a vapour
Sy contiou?l
verithing Mnever andienereffenushingled the met ary 
Epoch 32. Loss: 0.106233289508
At #e come flalound my upsitive. Then ureceered medade had a sut? a srace to mo to the three dimension this raressed I was
des more, and thoigh follod sevent of the sun was unsithitligg
inte meat was thick to astining. The mack as mo
modeld ardoot by ceetine
that illownyed him not soof suve somingly we clisted cammoethover
paredess be im
the tondense beramy manished in lrove lited overy limentions of Spacious that the
si

A doinco wit his rige, dinnt of fire in vellodidgnty, and petight. But im,
as in my eats. Then beillis puttreaded hepperaie uprepore, had at of
melbswining
ond. I fainte--laspeas all kind was beon. He und woudd war stelling. Sur mo the spoich lace a more byon peequl) with them the memical and but theck have an fouracl at the
grove sove for the
cabling for a micule that it not,' suid into modeck had so
higs and the sunion wing thore was dosning it
usonn our ob this mending
buanes grilding. I danis seeded to m
Epoch 47. Loss: 0.101682383206
A doo "ot? bains and shiothers and the stouch a little ffilk and grey
fein there was a broupf full give yolfing, dravely foruthen the haud wathered hove. un imptoby drimp and lact he man to fvar
the appour fire say in frane shath. Hight. Time sawns mealt the Dravedlarnzltiref and rguntelling and. He adembyted filicind torvadist shopled mit
ingar of clowor Saster--y coon a caming thinghor,'
said of bering furn--very piring wirh mare thee were and has a

Epoch 62. Loss: 0.0980828907029
At outur-ou hall, an
a sigud renn; con, shoilt wigh the promyss baid int, into the upsemed fade revelte;. I taluss firghing I recale in ome ponding the willes ge mach a sival the mo minter. Yot in the
s uil you wast to mplived the jem to me brighingst ingerain of he was ftont ssive it a conded on a pramo--stalos me. O*
iromnhtt to the other. I looked round mo. I wooked round me. I was on what
seemed to me and over tois of mpacing persain wirhingsed ty take hand the some was save had cruaghing, I wascendey on
Epoch 63. Loss: 0.0964944619502
A doinvedd,'s
intiricatered, in this onereaty oor whitedenge. I
Atmers by my hands of one huth oin herd, and thoight that to ascoursedoty hindedsy, and pety in of in chemu
pridey this toowher to
cunting the
whole doznners of iloty hairheng
and and of the facoonser. Yol chugmbenings ie vill--of him it is an ang rame erilour op sight
apaut, and the
wholl I ins twent le seemed to the tull of the neto
rexulare mon itore pe

A countiners the apporetting that wazl af at me. I po
mestrane hom a mover ore of them
w'On and firs wnow no rape a sample simes an and agoounily poruad onjured to me shaconstinn--ganding in the indellevery corturding that it moued reme
thoug. He ame howd, toll
shoulders of light
hily up it this in a pored undaksed. I net seen dame soness on whicns were lown of lo8king of my rof peat?' uropotile that hazg, said the thing wht icory it the greuld to expickea clear in ackipan thisk fill gencald Iimed the Seel o
Epoch 78. Loss: 0.102138687657
And, waw harel' Horerext.
AForesov,'

hir_. Yould
hThed?'CAhrof lormalk.

orewwhing.'
rroreq! th
ing
ouchTlurs,o I
fover
bGFarext absinxpparkparoulwathar,
whu_s ofrewh dredlbor.'
Helonexnd.'Yred. Wed opj4Thtr Dimuthals
wny pomeredb-whry abors.

Tus of thith,'shallicarme, axex
Thorrgally.
Hnorys, Bunexd th
drancu folly
exf Morm.'Horewhy

wpoinathry persmaked. TItrrglin,. Hably warcin
rearh, Hory Yoom..
HYouchor awyon,


Thinx, torJrawh?'A
wot lor?
Yfor

Epoch 93. Loss: 0.0876210788923
A doink

'mttens that would sevelt on might
of a prownerd ard I had come into rece a to vourd
-fowree jounticull pather were sondule, bst cor aidoreay legothor a moment. Are illist into a
kinct te exprace wis nets of white stoack. Then say it for a little tracky-sinaly panned the Time Machine
?'s word I remorkedef the sur hapred him llope in mush the treary that if fruched mereal of them. and spalk?' has tring for a moment I fall experse
he gat of thoune hime sounded?'T ailinger
antiles might to meder the li
